# Foundations of Durable AI with Temporal

In this workshop, we'll build toward creating AI agents, but let's start with a simple chain:

Use an LLM to generate research -> then produce a PDF from that research.

### Notebook Setup

This is a hands-on workshop!

All of the instructors slides and code samples are are executable in the workshop notebooks.
We encourage you to follow along and play with the samples!

At the end of every chapter (notebook) will be a hands-on lab.
This a self-guided experience where the instructor gives a prompt (not an llm haha) with a notebook and some starter code and the attendees solve the puzzle.

We are going to create a Research application that makes a call to the OpenAI API, conducts research on a topic of your choice, and generates a PDF report from that research. Let's go ahead and first set up your notebook.

### Create an `.env` File

Next you'll create a `.env` file to store your API keys.
In the file browser on the left, create a new file and name it `.env`.
Then double click on the `.env` file and add the following line with your API key.

```
LLM_API_KEY = YOUR_API_KEY
LLM_MODEL = "openai/gpt-4o"
```

By default this notebook uses OpenAI's GPT-4o.
If you want to use a different LLM provider, look up the appropriate model name [in the documentation](https://docs.litellm.ai/docs/providers) and change the `LLM_MODEL` field and provide your API key.

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

# Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings

import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

### Let's Prompt the LLM

Our application will make an LLM call.

In [ ]:
# Run this codeblock to load it into the program
from litellm import completion, ModelResponse

def llm_call(prompt: str) -> ModelResponse:
    response = completion(
      model=LLM_MODEL,
      api_key=LLM_API_KEY,
      messages=[{ "content": prompt,"role": "user"}]
    )
    return response

prompt = "Give me 5 facts about elephants."
result = llm_call(prompt)["choices"][0]["message"]["content"]

print(result)

### Making It Interactive

Great! You've successfully made your first LLM call with a hardcoded prompt. Now, let's transform this into an interactive research assistant that will perform research on any topic of your choosing.

This is the first step toward building our Gen-AI application.

In [ ]:
# Run this codeblock to load it into the program
# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call(prompt)

# Extract the response content
response_content = result["choices"][0]["message"]["content"]

print("Research complete!")
print("-"*80)
print(response_content)

### Let's Put our Research Results into a PDF

Once you have your research data, you’ll call an action that takes the results and writes it out to a PDF.

In [ ]:
# This function takes text content and formats it into a professional-looking PDF document.
# Run this codeblock to load it into the program.

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

def create_pdf(content: str, filename: str = "research_report.pdf") -> str:
    doc = SimpleDocTemplate(filename, pagesize=letter)

    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))

    paragraphs = content.split('\n\n')
    for para in paragraphs:
        if para.strip():
            p = Paragraph(para.strip(), styles['Normal'])
            story.append(p)
            story.append(Spacer(1, 12))

    doc.build(story)
    return filename

create_pdf("Hello PDF!", filename="test.pdf")

### Open the PDF

Download the `test.pdf` PDF and open it by following these steps:
  - Right-click the PDF file in the file explorer
  - Select "Download"
  - Open it locally on your machine

You should see a title **Research Report** and the words **Hello PDF!** in the document.

### Bringing it all Together

You now have multiple functions that you can execute to achieve a task.
Next, write the code to bring this all together:

1. Use the LLM to respond to a prompt
2. Call the `create_pdf` action to create a PDF with the response to your prompt.

In [ ]:
# Step 1: Call the `llm_call` function with `prompt` as an argument.
# Step 2: Call the `create_pdf` function with `response_content` as the first argument and
# the file name you want to call your PDF (as a string) as the second argument.
# Step 3: Run the code block to execute the program.
# Step 4: Download your PDF and open it by following these steps:
#  - Right-click the PDF file in the file explorer
#  - Select "Download"
#  - Open it locally on your machine

# Make the API call
print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ")
result = llm_call() # TODO: Call the `llm_call` function with `prompt` as an argument.

# Extract the response content
response_content = result["choices"][0]["message"]["content"]

pdf_filename = # TODO: Call the `create_pdf` function that takes in our 2 arguments: response_content and the file name you want to call your PDF (as a string).
print(f"SUCCESS! PDF created: {pdf_filename}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_01_Foundations_of_Durable_AI_with_Temporal" / "pdf_generation_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Instructor-Led Demo (Expand for instructor notes or to run on your own)
<!--
Normal Execution Demo:
1. To demonstrate the power of durable execution, we'll first show the power of running the app with no durable execution.
2. The instructions will also be in the README at the `demos` level. Follow the `Setup` step first before running.
3. From the `demos/module_one_01_foundations_aiic_loop/app.py` directory, run `app.py` with `python app.py`.
4. When prompted, provide the prompt you want to prompt OpenAI in the command line.
5. Before the process generates a PDF, kill the process.
6. Rerun the application again with `python app.py` and show that the process restarted and you have to have your application start the research again. Emphasize that from a cost perspective, this could be very costly, because you could have to re-run through many tokens to get to where you left off.

Durable Execution Demo:
1. Now show the durable version by switching into the ``demos/module_one_02_adding_durability` directory.
2. Run the Worker with `uv run worker.py`.
3. Run the Workflow with `uv run starter.py`.
4. When prompted, provide the prompt you want to prompt OpenAI in the command line.
5. Before the process generates a PDF, kill the Worker.
6. Rerun the Worker and show that you continue right where you left off.
7. Emphasize that you lost no progress or data. The Workflow will continue by generating the PDF (available in the same directory) and completing the process successfully.
10. Show the Workflow Execution completion in the Web UI.
-->

---

# Exercise 1 - Adding More Actions

* In this exercise, you'll:
  * Call actions with your GenAI Application
  * Extract structured information from LLM responses to coordinate between different actions.
* Go to the **Exercises** Directory and open the **01_Foundations_of_Durable_AI_with_Temporal** Directory
* Open the _Practice_ directory and follow the instructions and filling in the `TODO` statements and running the codeblocks.
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers

### What's Next?

This chapter introduced you the importance of durability in GenAI applications. Further your learning with these resources:

### Resources

- [A mental model for agentic AI Applications blogpost](https://temporal.io/blog/a-mental-model-for-agentic-ai-applications)
- [From AI hype to durable reality — why agentic flows need distributed-systems discipline blogpost](https://temporal.io/blog/from-ai-hype-to-durable-reality-why-agentic-flows-need-distributed-systems)